## Post-deployment QA: Reply tool Instrumentation - New VEFU Events 

This task is about ensuring that the newly added events to the [VisualEditorFeatureUse](https://meta.wikimedia.org/wiki/Schema:VisualEditorFeatureUse) schema are being logged correctly in database.

Test scenarios and status are documented in this [QC document](https://docs.google.com/spreadsheets/d/1txtypYKZHCiZEFnkye9LX7riF3VEippMmvN6ILs_BUQ/edit#gid=605234531)

# Contents

1. [Advanced Affordance Events](#1.-Advanced-Affordance-Events)
2. [Dialog Prevent Show Event](#2.-Dialog-Prevent-Show-Event)
3. [Special Characters Toolbar Events](#3.-Special-Characters-Toolbar-Events)

    

In [3]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); library(glue); library(lubridate); library(scales)
})

In [6]:
# Collect VEFU events after deployment
query <- 
"SELECT
    date_format(dt, 'yyyy-MM-dd') AS event_dt,
    wiki AS wiki,
    event.editor_interface AS editor_interface,
    event.platform AS platform,
    event.feature As feature,
    event.integration AS integration,
    event.action AS action,
    COUNT(*) AS events
FROM event_sanitized.visualeditorfeatureuse
WHERE 
-- reviewing a few days prior to confirm when events started firing
    year = 2020 
    AND month >= 09
    AND wiki <> 'testwiki'
GROUP BY
    date_format(dt, 'yyyy-MM-dd'),
    wiki,
    event.editor_interface,
    event.feature,
    event.integration,
    event.platform,
    event.action
"

In [7]:
collect_vefu_events <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [8]:
collect_vefu_events$event_dt <- as.Date(collect_vefu_events$event_dt, format = "%Y-%m-%d")

# 1. Advanced Affordance Events

Event added that logs when people "open" the Reply Tool's advanced "shelf". 

This functionality can be accessed by clicking the Advanced link that will appear beneath the text input area and can be used to customize the edit summary that accompanies the comments they post with the Reply Tool.

It was deployed on 17 September 2020 to all wikis where the reply tool was accessible. 

**Event**	**Name**
* An event that fires when the Advanced link/affordance is clicked to "show" the Comment summary field	feature: dtReply, action: advanced-show
* An event that fires when the Advanced link/affordance is clicked to "hide" the Comment summary field	feature: dtReply, action: advanced-hide


[Task](https://phabricator.wikimedia.org/T261816)

## Confirm Events Firing as Expected

In [6]:
advanced_show_events <- collect_vefu_events %>%
    filter(action == 'advanced-show') %>%
    arrange(date)

advanced_show_events 

date,wiki,editor_interface,platform,feature,integration,action,events
<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2020-09-17,arwiki,visualeditor,desktop,dtReply,discussiontools,advanced-show,1
2020-09-17,hewiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,frwiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,kowiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-18,zhwiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-19,enwiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-19,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-show,1
2020-09-20,huwiki,visualeditor,desktop,dtReply,discussiontools,advanced-show,1


Events are being recorded as early as 17 September 2020, which was the date of deployment. 

Confirmed that advanced show events are only being recorded for dtreply feature and discussiontools integration as expected. Events are recorded for both wikitext and visualeditor.

In [7]:
advanced_close_events <- collect_vefu_events %>%
    filter(action == 'advanced-hide') %>%
    arrange(date)

advanced_close_events 

date,wiki,editor_interface,platform,feature,integration,action,events
<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2020-09-17,arwiki,visualeditor,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-17,hewiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-18,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-18,zhwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-19,jawiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-20,huwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,2
2020-09-21,arwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,1
2020-09-28,svwiki,wikitext,desktop,dtReply,discussiontools,advanced-hide,5
2020-09-29,enwiki,visualeditor,desktop,dtReply,discussiontools,advanced-hide,6


Confirmed there are fewer advanced hide events compared to advanced show events as expected. Advanced hide events also start coming in on 17 September 2020 (date of deployment).

Confirmed that advanced hide events are only being recorded for dtreply feature and discussiontools integration as expected. Events are recorded for both wikitext and visualeditor.

## Total Events

Review total events to confirm there are fewer advanced-hide events to advanced-show events.

In [8]:
total_advanced_events <- collect_vefu_events %>%
    filter(action == 'advanced-hide' |
          action == 'advanced-show') %>%
    group_by(action)  %>%
    summarise(total_events = sum(events))

total_advanced_events 

`summarise()` ungrouping output (override with `.groups` argument)



action,total_events
<chr>,<int>
advanced-hide,71
advanced-show,142


Confirmed that there are more advanced show events compared to advanced hide. As of 26 Oct 2020, there have been 142 advanced-show events and 71 advanced-hide events (213 advanced affordance events total)

There are no events in any other platform other than "Desktop" since this feature is only available on desktop.

## Event Counts by Wiki

In [9]:
total_advanced_events_bywiki <- collect_vefu_events %>%
    filter(action == 'advanced-hide' |
          action == 'advanced-show') %>%
    group_by(action, wiki)  %>%
    summarise(total_events = sum(events)) %>%
    arrange(wiki)

total_advanced_events_bywiki

`summarise()` regrouping output by 'action' (override with `.groups` argument)



action,wiki,total_events
<chr>,<chr>,<int>
advanced-hide,arwiki,7
advanced-show,arwiki,14
advanced-hide,cswiki,2
advanced-show,cswiki,3
advanced-hide,enwiki,6
advanced-show,enwiki,7
advanced-hide,enwikiversity,1
advanced-show,enwikiversity,1
advanced-hide,eswiki,1


Advanced events recorded for arwiki, cswiki, enwiki, frwiki, hewiki, huwiki, jawiki, kowiki, nlwiki, svwiki, viwiki, and zhwiki. A few observations:
* For all wikis, there are more advanced-show events then advanced-hide events as expected.
* For viwiki and kowiki, there is only one advanced-show event recorded (no advanced-hide event).
* Enwiki and HeWiki have events but are not listed on the deployment wikis; however, these are likely wikis where it was enabled for testing using ?dtenable=1.


In [10]:
total_advanced_events_bywiki_both <- collect_vefu_events %>%
    filter(action == 'advanced-hide' |
          action == 'advanced-show') %>%
    group_by( wiki)  %>%
    summarise(total_events = sum(events)) %>%
    arrange(total_events)

total_advanced_events_bywiki_both 

`summarise()` ungrouping output (override with `.groups` argument)



wiki,total_events
<chr>,<int>
metawiki,1
viwiki,1
enwikiversity,2
hewiki,2
itwiki,2
kawiki,2
plwiki,2
ptwiki,2
eswiki,3


Svwiki and Nlwiki have the largest number of advanced affordance events recorded to date (55 for svwiki and 32 for nlwiki).

# 2. Dialog Prevent Show Event 
Event added that is fired when someone attempts to switch from the Reply Tool's source to visual mode after drafting a comment that contains any combination of the following syntax: extension, template or table.

This is associated with the editor_switch feature and discussiontools integration.

[Task](https://phabricator.wikimedia.org/T259673)

## Confirm events firing as expected

In [44]:
switch_prevent_events <- collect_vefu_events %>%
    filter(action == 'dialog-prevent-show') %>%
    arrange(desc(date))

switch_prevent_events

date,wiki,editor_interface,platform,feature,integration,action,events
<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
2020-11-10,nlwiki,wikitext,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-11-09,nlwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,2
2020-11-03,arwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-30,plwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-25,arwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-16,arwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-14,frwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,1
2020-10-13,nlwiki,visualeditor,desktop,editor-switch,discussiontools,dialog-prevent-show,2


Since the deployment date, only 7 dialog-prevent-show events have been recorded. Many of these are likely testing events. It appears that this dialog appears very rarely based on the current data. 

All events are for the editor_switch feature and discussion tools integration as expected. The editing interface for these events are recorded as 'visual editor' - Is that correct given that this action is supposed to record when an editor is prevented from switching to visual mode from source?

Logged an event on Fr.Wiki, cswiki, and kowiki. May take some time for event to come in. 

# 3. Special Characters Toolbar Events

Events added to collect data for when people are using the visual editor and wikieditor special characters toolbar. Note: Support for this feature has not yet been added for the reply tool. This instrumentation was added to inform how frequently users are using the special character's toolbar in other editing interfaces.

New events deployed on 8 October 2020
[Task](https://phabricator.wikimedia.org/T264146)

## Events by date

In [11]:
special_character_events_bydate <- collect_vefu_events %>%
    filter(feature == 'specialCharacter') %>%
    arrange(event_dt)

head(special_character_events_bydate)

,event_dt,wiki,editor_interface,platform,feature,integration,action,events
,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,2020-10-13,mediawikiwiki,visualeditor,desktop,specialCharacter,page,dialog-abort,1
2,2020-10-13,mediawikiwiki,visualeditor,desktop,specialCharacter,page,insert-latin,1
3,2020-10-13,mediawikiwiki,visualeditor,desktop,specialCharacter,page,window-open-from-tool,1
4,2020-10-14,mediawikiwiki,visualeditor,desktop,specialCharacter,page,window-open-from-tool,1
5,2020-10-14,mediawikiwiki,visualeditor,desktop,specialCharacter,page,dialog-abort,1
6,2020-10-15,enwiki,visualeditor,desktop,specialCharacter,page,dialog-abort,1


We first start recording events on 13 October 2020 as expected

## Events by action

In [14]:
special_character_events_byaction <- collect_vefu_events %>%
    filter(feature == 'specialCharacter') %>%
    group_by(action)  %>%
    summarise(num_events = sum(events)) %>%
    arrange(desc(num_events))

special_character_events_byaction

`summarise()` ungrouping output (override with `.groups` argument)



action,num_events
<chr>,<int>
window-open-from-tool,6744
dialog-abort,5887
insert-other,4994
insert-latin,1646
insert-symbols,1267
insert-cyrillic,817
insert-ipa,699
insert-bangla,405
insert-greek,402


'Dialog-abort' and 'window-open-tool' are the most frequently recorded SpecialCharacter toolbar events. There are more events opening the special characters toolbar vs closing the toolbar as expected.

There are 21 different insert actions for when someone adds a character from the toolbar. `insert-other`, `insert-latin`, and `insert-symbols` are the most frequently used options. `insert-gujarati`, `insert-tamil`, and	`insert-sinhala`	are the least used actions. 



## Events by Platform, Integration and Editor Interface

In [15]:
special_character_events_bytype <- collect_vefu_events %>%
    filter(feature == 'specialCharacter') %>%
    group_by(platform, integration, editor_interface)  %>%
    summarise(num_events = sum(events)) %>%
    arrange(desc(num_events))

special_character_events_bytype

`summarise()` regrouping output by 'platform', 'integration' (override with `.groups` argument)



platform,integration,editor_interface,num_events
<chr>,<chr>,<chr>,<int>
desktop,page,visualeditor,20924
desktop,page,wikitext-2017,2875


All special character events recorded since the instrumentation was deployed in October 2020 have been on desktop using both the visualeditor (VE in visual mode) or wikitext-2017 (VE in source mode) as expected. 

All special character events are recorded as main edit type events (`event.integration = 'page')`. This is also expected behavior as this instrumentation has not been deployed to the discussion tools. 



## Events by Wiki

In [19]:
special_character_events_bywiki <- collect_vefu_events %>%
    filter(feature == 'specialCharacter') %>%
    group_by(wiki)  %>%
    summarise(num_events = sum(events)) %>%
    arrange(desc(num_events))

head(special_character_events_bywiki)

`summarise()` ungrouping output (override with `.groups` argument)



wiki,num_events
<chr>,<int>
ruwiki,3261
enwiki,3191
plwiki,2566
frwiki,2030
dewiki,1456
eswiki,1304


In [18]:
n_distinct(special_character_events_bywiki$wiki)

[1] 122

We've recorded special character toolbar events on 122 different wikis. The most events were recorded for Russian Wikipedia, English Wikipedia, Polish Wikipedia, French Wikipedia, and German Wikipedia. 

Below is breakdown of the special character events by action for each of these wikis.

In [35]:
## Review the number of special character events by action for each of the top 6 wikis with the most recorded overall events
special_character_events_bywikiaction <- collect_vefu_events %>%
    filter(feature == 'specialCharacter',
            wiki %in% c('ruwiki', 'enwiki', 'plwiki', 'frwiki', 'dewiki', 'eswiki')) %>%
    group_by(wiki, action)  %>%
    summarise(num_events = sum(events)) %>%
    arrange(desc(wiki, num_events))

special_character_events_bywikiaction

`summarise()` regrouping output by 'wiki' (override with `.groups` argument)



wiki,action,num_events
<chr>,<chr>,<int>
ruwiki,dialog-abort,873
ruwiki,insert-arabic,14
ruwiki,insert-arabicextended,1
ruwiki,insert-bangla,2
ruwiki,insert-canadianaboriginal,13
ruwiki,insert-cyrillic,30
ruwiki,insert-greek,34
ruwiki,insert-greekextended,13
ruwiki,insert-hebrew,61


## Check to make sure specialCharacter specific features are not recorded elsewhere

In [26]:
insert_type_events <- collect_vefu_events %>%
    filter(grepl('^insert', action)) %>%
    group_by(feature)  %>%
    summarise(num_events = sum(events)) %>%
    arrange(desc(num_events))

insert_type_events

`summarise()` ungrouping output (override with `.groups` argument)



feature,num_events
<chr>,<int>
table,104404
specialCharacter,11168


Insert type actions are recorded only for specialCharacter and table features as expected.  